In [1]:
import pandas as pd
import numpy as np
import gzip
import subprocess
import statsmodels.api as sm

# Read in data

In [54]:
out_dir = "GeneExpressionFiles/"
dict_genes = {"KDSR":"ENSG00000119537.9", "LARS2":"ENSG00000011376.5", "CSF1R":"ENSG00000182578.8", \
	"PDGFRB": "ENSG00000113721.7", "TIGD6":"ENSG00000164296.6", "RPS20P4":"ENSG00000241907.1", \
		"RPL7P1":"ENSG00000214485.5", "HMGXB3":"ENSG00000113716.8", "SLC26A2":"ENSG00000155850.7", \
			"C1orf198":"ENSG00000119280.12", "COG2":"ENSG00000135775.8", "C1D":"ENSG00000197223.7", \
			"CXCR2P1": "ENSG00000229754.1", "RUFY4":"ENSG00000188282.8", "TNS1":"ENSG00000079308.11", \
			"LIMD1":"ENSG00000144791.5", "SEMA3A":"ENSG00000075213.5", "ZNF572":"ENSG00000180938.5", \
			"RP11-1082L8":"ENSG00000243150.1", "LOH12CR2":"ENSG00000205791.2", "MANSC1":"ENSG00000111261.8", \
			"BORCS5":"ENSG00000165714.6", "LRP6":"ENSG00000070018.4", "BCL2":"ENSG00000171791.9", \
			"RPL9P32":"ENSG00000242100.1", "URI1":"ENSG00000105176.12", "SSTR3":"ENSG00000183473.5", \
			"IL2RB":"ENSG00000100385.8", "C1QTNF6":"ENSG00000133466.9", "KCTD17":"ENSG00000100379.13", \
			"Z82188":"ENSG00000255594.1", "TMPRSS6":"ENSG00000187045.12", "RP1-151B14.6":"ENSG00000235237.1",\
			"RAC2":"ENSG00000128340.10", "DCHS2":"ENSG00000197410.6", "PLRG1":"ENSG00000171566.7", \
			"RP11-158C21.3":"ENSG00000248611.1", "TPO":"ENSG00000115705.15", "VWF":"ENSG00000110799.9"}
pop_dict = {"KDSR":["YRI"], "LARS2":["CEU", "FIN", "GBR"], "CSF1R":["FIN", "GBR"]}

### Define dictionary of STRs with their corresponding Gene names

In [46]:
dict_STR_gene= {"CSF1PO":["CSF1R", "PDGFRB", "TIGD6", "RPS20P4", "RPL7P1", "HMGXB3", "SLC26A2"], \
	"D1S1656": ["C1orf198", "COG2"], \
	"D2S441": ["C1D"], \
	"D2S1338": ["CXCR2P1", "RUFY4", "TNS1"], \
	"D3S1358": ["LIMD1", "LARS2"], \
	"D7S820": ["SEMA3A"], \
	"D8S1179": ["ZNF572", "RP11-1082L8"], \
	"D12S391": ["LOH12CR2", "MANSC1", "BORCS5", "LRP6"], \
	"D18S51": ["BCL2", "KDSR"], \
	"D19S433": ["RPL9P32", "URI1"], \
	"D22S1045": ["SSTR3", "IL2RB", "C1QTNF6", "KCTD17", "Z82188", "TMPRSS6", "RP1-151B14.6", "RAC2"], \
	"FGA": ["DCHS2", "PLRG1", "RP11-158C21.3"], \
	"TPOX": ["TPO"], "VWA":["VWF"]
}

In [ ]:
# FROM BASH TO GET THE LINE OF GENE WITH gene_id
# zcat gencode.v12.annotation.gtf.gz| grep -w CSF1R | awk -F '\t' '$3 == "gene" {print $0}'

## Extract Gene Expression Data from File According to Transcript Name

In [55]:
# dict_genes = {"CSF1R":"ENSG00000182578.8", "TIGD6":"ENSG00000164296.6", "C1D":"ENSG00000197223.7", \
# 	"LARS2":"ENSG00000011376.5", "KDSR":"ENSG00000119537.9", "PLRG1":"ENSG00000171566.7" }

genes_df = {}

# Get header
with gzip.open("../GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.gz", "rt") as exp:
	header_read = False
	for line in exp:
		if not header_read:
			genes_df["ID"] = line.rstrip('\n').split("\t")
			header_read = True
		break

# Extract gene lines from main file
for key in dict_genes:
	command = "zcat ../GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.gz | grep " + dict_genes[key]
	# Remember to include universal_newlines to make sure that bites are not used
	p = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True).communicate()[0]
	#print(p.split('\t'))
	gene_info = p.rstrip('\n').split("\t")
	if len(gene_info) == 0:
		print("Did not find " + key)
	genes_df[key] = gene_info

# for key in genes_df:
# 	print(key)
# 	print(len(genes_df[key]))


In [29]:

# for key in genes_dataframe:
# 	if ini:
# 		current_len = len(genes_dataframe[key])
# 		ini = 0
# 		continue
# 	else:
# 		next_len = len(genes_dataframe[key])
# 		if current_len == next_len:
# 			print("YES")
# 		else:
# 			print("NO")
# 			print(key)


['']


### Number of samples in the following dataframe is without yet intersecting with available STR genotypes

In [56]:
df_geneExp = pd.DataFrame.from_dict(genes_df)
df_geneExp = df_geneExp.iloc[4: , :].reset_index(drop=True) # Gettiung rid of unnecesary data
df_geneExp

df_geneExp.to_csv("../GeneExpressionFiles/CumulativeSignificant.txt", sep="\t", index=False)

## Load 1000 Genomes Population IDs. Intersect samples with available gene expression for Cumulative Analysis

In [57]:
k_pops = pd.read_csv("../1000Genomes_samples.tsv", sep="\t")

# #Sample name and population code
k_pops.columns

""" k_pops["Sample name"]
contains the 1000 genomes IDs for finding intersecrtion of both datasets
"""

' k_pops["Sample name"]\ncontains the 1000 genomes IDs for finding intersecrtion of both datasets\n'

In [58]:
# Get IDs from 1000 genomes Sample name and see how many of those are present in gene expression data set
# We shoudl get back the same number of individuals since the expression data is based on the 1000 genomes dataset
samples_KG = k_pops["Sample name"].to_numpy()
Final_GeneExp = df_geneExp.iloc[np.where(df_geneExp.ID.isin(samples_KG))]
Final_GeneExp

,ID,KDSR,LARS2,CSF1R,PDGFRB,TIGD6,RPS20P4,RPL7P1,HMGXB3,SLC26A2,...,KCTD17,Z82188,TMPRSS6,RP1-151B14.6,RAC2,DCHS2,PLRG1,RP11-158C21.3,TPO,VWF
0,HG00096,9.82025168177286,9.75294665719534,0.22535561567989,0.222846270992758,2.11627034895555,0.192236333516375,2179.49523786123,16.7437343809641,1.60924036151234,...,8.30019136588444,110.989632239119,0.300164628624847,0.128995516671278,304.968148057172,0.102627947650203,34.3533103291144,0.0637852380013456,0.0560387443427862,0.0134671047637939
1,HG00097,7.50393757785698,9.81141124697793,0.2192694444275,0.390644915951969,1.13188912466145,0.205328197890487,2122.52771165654,13.2430444485235,1.66594468238588,...,4.08096909604071,82.3846879781457,0.127679045755552,0.204701243857849,289.085150298259,0.183705696840287,30.9352706202084,0.0057659014893724,-0.00514455941459295,0.00934618254588752
2,HG00099,5.56445760871555,5.38665152891203,0.17947283908112,0.141198017666443,1.01451663045898,0.00214132715694101,1392.55195252353,7.97390854631152,1.26355245470069,...,2.46084236771005,93.9249513726375,0.158571847263323,0.0660589613533657,153.443332347446,0.841411216686904,15.5515016220526,0.033892925214387,0.0392930273711004,0.00221758256008041
3,HG00100,7.55012341117795,10.9093477500582,0.223995300784694,0.702281030985763,2.45842158524617,0.0115254536964854,2042.15237522519,16.5283317000305,1.57768673782728,...,6.36872322410568,145.002779101021,0.0697133216209153,0.0786159991279376,308.703965690327,0.0615848232752146,37.7038988724454,-0.00664206749977456,-0.00171260457576453,0.013768776076553
4,HG00101,7.26794934063688,11.6940691012637,0.322167239293219,1.18846394604633,1.55979896204861,0.165185311165317,2044.7459504603,17.2897821852552,1.70172198770273,...,4.62096004381836,104.266366004897,0.281133126173038,0.126898297657425,305.787210197091,0.0719945752716314,34.0376793566019,0.0177231925703957,0.0337441975078422,0.0177746809531517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,NA20815,8.48687430577015,6.46914274214578,0.182468406026515,0.332987326049915,1.98098755278034,0.407187647773674,1771.88205924753,17.9844862159887,1.8905608849386,...,3.60025714432779,170.647680794753,0.481437034065924,0.17987628917033,305.548740712535,-0.145398618904338,26.4993545394434,0.0220895030611271,0.0718707826289235,0.0103359628192047
458,NA20816,9.85092742905656,7.68830411608624,0.12898771449886,0.0129976395200972,1.46859454991751,0.037808658121562,1953.05250258309,17.581287353207,1.46125013626133,...,5.78757632646451,150.087660374652,-0.0478187206727129,0.0709336414447974,308.386788304444,-0.0625725553541458,40.6785431770121,0.0399348738255154,0.0191270462935817,0.0724350912918395
459,NA20819,7.43752723529614,7.25220781958196,0.210543759722848,0.443625418800579,1.95656614950652,-0.0353994104363097,1925.1749989802,10.9677889030441,1.70891489603259,...,5.17388309739231,157.973864278234,-0.0348546430554069,0.354203931569191,276.048079238225,0.0938923460331484,36.5282219721215,-0.00426967352016453,0.00585462160928122,0.0115355906874427
460,NA20826,7.21705559505251,9.25964792004697,0.277418486715307,0.636912983778989,1.59648833766172,-0.0171399279986165,2124.42974215972,17.8435522633143,1.28054985520896,...,5.78961956434026,78.6670200390931,0.257690947994866,0.436910829606453,281.753035820741,0.292088722606422,33.1956884567349,0.102479578219417,0.109980342819786,0.0172504348600933


In [7]:
# Regressions function
def regression_analysis(str_genos, exp):
	""" Performs Regression_Analysis 
		for STR genotypes (X) and simulated gene expression (Y)
		and returns p-value and correlation coefficient
	"""
	mod = sm.OLS(exp, str_genos)
	fii = mod.fit()
	print(fii.summary())
	p_value = fii.summary2().tables[1]['P>|t|'] 
	r = fii.rsquared

	return p_value, r

### Load Beta Values and find intersection with Gene Expression Cumulatively

In [59]:
str_directory = "../1000Genomes_Beta_Genotypes/"
forestPlot_directory = "../ForestPlotData/"

In [ ]:
fname = str_directory + "CSF1PO_Mem_Genotypes.txt"
CSF1PO = pd.read_csv(fname, sep="\t", skiprows=1, names=['ID', 'Beta'])
CSF1PO.dropna(subset=['Beta'], inplace=True)
CSF1PO

In [10]:
# Get IDs by Population from 1000 Genomes Data
YRI = k_pops.loc[k_pops["Population code"] == "YRI"]
YRI = YRI['Sample name'].to_numpy()
CEU = k_pops.loc[k_pops["Population code"] == "CEU"]
CEU = CEU['Sample name'].to_numpy()
GBR = k_pops.loc[k_pops["Population code"] == "GBR"]
GBR = GBR['Sample name'].to_numpy()
FIN = k_pops.loc[k_pops["Population code"] == "FIN"]
FIN = FIN['Sample name'].to_numpy()
TSI = k_pops.loc[k_pops["Population code"] == "TSI"]
TSI = TSI['Sample name'].to_numpy()

In [51]:
from sklearn.linear_model import LinearRegression
from scipy import stats # For p value

In [53]:
compare = pd.merge(CSF1PO, Final_GeneExp, on='ID')
compare['CSF1R'] = compare['CSF1R'].astype(float)
compare['Beta'] = compare['Beta'].astype(float)

x = compare['Beta'].to_numpy()
y = compare['CSF1R'].to_numpy()

slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
print(r_value) # Pearson correlation 
print(p_value)



-0.11058705816825418
0.032982863042086794


In [40]:
compare['CSF1R'].shape

(372,)

In [136]:
# Get YRI samples adn beta genotypes from CSF1PO
CSF1PO_YRI = Final_GeneExp.iloc[np.where(Final_GeneExp.ID.isin(YRI))]
CSF1PO_YRI
# Get gene expression for CSF1R and ID
#CSF1R = df_geneExp[['ID', 'CSF1R']]
# Merge gene expression and genotypes
compare = pd.merge(CSF1PO_YRI, CSF1PO, on='ID')
compare['KDSR'] = compare['KDSR'].astype(float)
compare

,ID,KDSR,Beta
0,NA18486,8.944139,38.420
1,NA18488,8.872616,35.550
2,NA18489,7.767522,37.795
3,NA18498,8.665523,35.340
4,NA18499,9.568111,37.630
...,...,...,...
65,NA19236,9.268054,38.825
66,NA19247,6.858720,29.460
67,NA19248,9.806799,40.440
68,NA19256,7.400542,35.905


In [137]:
pval, rsqrd = regression_analysis(compare['Beta'].tolist(), compare['KDSR'].tolist())
print(pval)
print(rsqrd)

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.981
Model:                            OLS   Adj. R-squared (uncentered):              0.980
Method:                 Least Squares   F-statistic:                              3516.
Date:                Mon, 08 Aug 2022   Prob (F-statistic):                    6.24e-61
Time:                        12:17:20   Log-Likelihood:                         -108.76
No. Observations:                  70   AIC:                                      219.5
Df Residuals:                      69   BIC:                                      221.8
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [8]:
YRI = k_pops.loc[k_pops["Population code"] == "YRI"]
# LARS2_CEU = k_pops.loc[k_pops["Population code"] == "CEU"]
# LARS2_FIN = k_pops.loc[k_pops["Population code"] == "FIN"]
# LARS2_GBR = k_pops.loc[k_pops["Population code"] == "GBR"]
# CSF1R_FIN = k_pops.loc[k_pops["Population code"] == "FIN"]
# CSF1R_GBR = k_pops.loc[k_pops["Population code"] == "GBR"]

In [118]:
# samples_YRI = YRI["Sample name"].to_numpy()
# samples_LARS2_CEU = LARS2_CEU["Sample name"].to_numpy()
# samples_LARS2_FIN = LARS2_FIN["Sample name"].to_numpy()
# samples_LARS2_GBR = LARS2_GBR["Sample name"].to_numpy()
# samples_CSF1R_FIN = CSF1R_FIN["Sample name"].to_numpy()
# samples_CSF1R_GBR = CSF1R_GBR["Sample name"].to_numpy()

YRI = k_pops.loc[k_pops["Superpopulation code"] == "AFR"]
CEU = k_pops.loc[k_pops["Superpopulation code"] == "EUR"]
ALL = k_pops.loc[(k_pops["Superpopulation code"] == "EUR") | (k_pops["Superpopulation code"] == "AFR")]

samples_YRI = YRI["Sample name"].to_numpy()
samples_CEU = CEU["Sample name"].to_numpy()
all_samples = ALL["Sample name"].to_numpy()



In [138]:
AMR = k_pops.loc[k_pops["Population code"] == "MXL"]
samples_AMR = AMR["Sample name"].to_numpy()
AMR_df = df.iloc[np.where(df.ID.isin(samples_AMR))]
AMR_df.to_csv("../GeneExpressionFiles/AMR.txt", sep='\t', index=False)

# Get intersection 

In [126]:
# # YRI 
# YRI_df = df.iloc[np.where(df.ID.isin(samples_YRI))]
# YRI_df = YRI_df[["ID", "KDSR"]]

# # LARS2 CEU
# LARS2_CEU_df = df.iloc[np.where(df.ID.isin(samples_LARS2_CEU))]
# LARS2_CEU_df = LARS2_CEU_df[["ID", "LARS2"]]

# # LARS2 FIN
# LARS2_FIN_df = df.iloc[np.where(df.ID.isin(samples_LARS2_FIN))]
# LARS2_FIN_df = LARS2_FIN_df[["ID", "LARS2"]]

# # LARS2 GBR
# LARS2_GBR_df = df.iloc[np.where(df.ID.isin(samples_LARS2_GBR))]
# LARS2_GBR_df = LARS2_GBR_df[["ID", "LARS2"]]

# # CSF1R_FIN
# CSF1R_FIN_df = df.iloc[np.where(df.ID.isin(samples_CSF1R_FIN))]
# CSF1R_FIN_df = CSF1R_FIN_df[["ID", "CSF1R"]]

# # CSF1R_GBR
# CSF1R_GBR_df = df.iloc[np.where(df.ID.isin(samples_CSF1R_GBR))]
# CSF1R_GBR_df = CSF1R_GBR_df[["ID", "CSF1R"]]

# YRI 
YRI_df = df.iloc[np.where(df.ID.isin(samples_YRI))]
YRI_df = YRI_df[["ID", "KDSR"]]

# LARS2 CEU
LARS2_CEU_df = df.iloc[np.where(df.ID.isin(samples_CEU))]

# ALL SAMPLES
ALL_df = df.iloc[np.where(df.ID.isin(all_samples))]


In [81]:
# YRI_df.to_csv("YRI_KDSR.txt", sep='\t', index=False)
# LARS2_CEU_df.to_csv("CEU_LARS2.txt", sep='\t', index=False)
# LARS2_FIN_df.to_csv("FIN_LARS2.txt", sep='\t', index=False)
# LARS2_GBR_df.to_csv("GBR_LARS2.txt", sep='\t', index=False)
# CSF1R_GBR_df.to_csv("GBR_CSF1R.txt", sep='\t', index=False)
# CSF1R_FIN_df.to_csv("FIN_CSF1R.txt", sep='\t', index=False)

In [105]:
YRI_df.to_csv("../GeneExpressionFiles/YRI.txt", sep='\t', index=False)
LARS2_CEU_df.to_csv("../GeneExpressionFiles/CEU.txt", sep='\t', index=False)

In [128]:
ALL_df.to_csv("../GeneExpressionFiles/ALL.txt", sep='\t', index=False)


SyntaxError: invalid syntax (2525261656.py, line 1)

SyntaxError: invalid syntax (1466462207.py, line 2)